In [ ]:
import pandas as pd

df = pd.read_csv("baby_reviews_small.csv")
df.head()



,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,sentiment
0,A1OFPQMJWMM4XM,B002OHD3XS,"KARARED ""Kc""","[0, 0]",I love this booster. We have two and it's wel...,5,Love this booster....SAFETY FIRST!!!!!,1368403200,"05 13, 2013",1
1,A1Z94P3Q8O0D21,B00354UNSM,Jana Pettus,"[1, 1]","This has a million parts, is hard to wash, and...",1,Don't waste your money,1393891200,"03 4, 2014",0
2,A1V981XARKIMNT,B001KZJY2O,Chad and Kristen,"[1, 2]",This pillow probably cost less than a dollar t...,1,Terrible quality - do not buy this pillow,1367193600,"04 29, 2013",0
3,A1TCX0R35T4WEH,B000056J2D,"MomOf2BOYS ""Lucy""","[1, 2]","My 2 1/2 year olds highchair was terrible, so ...",5,Great booster,1115683200,"05 10, 2005",1
4,A1N54YHG0KQI91,B001PBFWMO,Toddler Mom,"[1, 1]",We got this when our daughter was only a few m...,5,We LOVE this!,1358899200,"01 23, 2013",1


Preparo todo para limpiar el texto y defino la función preprocess. Importo las librerías necesarias, cargo las stopwords y creo el stemmer. Luego la función limpia cada review eliminando HTML, pasando a minúsculas, quitando URLs y símbolos, removiendo stopwords y aplicando stemming. Devuelve la versión limpia del texto lista para usar en el modelo.

In [ ]:
import re
from bs4 import BeautifulSoup

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def preprocess(text):
    if not isinstance(text, str):
        return ""

    text = BeautifulSoup(text, "html.parser").get_text()

    text = text.lower()

    text = re.sub(r"http\S+|www\S+", " ", text)

    text = re.sub(r"[^a-z0-9\s]", " ", text)

    tokens = text.split()

    tokens = [t for t in tokens if t not in stop_words]

    tokens = [stemmer.stem(t) for t in tokens]

    return " ".join(tokens)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = df.dropna(subset=["reviewText"])


Separo los datos en entrenamiento (80%) y test (20%). Uso stratify=y para mantener el mismo equilibrio entre positivas y negativas en ambos conjuntos.

In [ ]:
from sklearn.model_selection import train_test_split

X = df["reviewText"]
y = df["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

len(X_train), len(X_test)


(3996, 1000)

Creo un pipeline que primero convierte el texto en vectores usando CountVectorizer (usando tu función preprocess) y luego entrena un modelo de regresión logística para clasificar las reviews según su sentimiento.



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

pipeline_lr = Pipeline([
    ("vect", CountVectorizer(
        preprocessor=preprocess,   # usamos tu función aquí
        ngram_range=(1, 2),        # unigrams + bigrams
        max_features=5000          # tamaño máximo de vocabulario
    )),
    ("clf", LogisticRegression(max_iter=1000))
])

pipeline_lr.fit(X_train, y_train)


Pipeline(steps=[('vect',
                 CountVectorizer(max_features=5000, ngram_range=(1, 2),
                                 preprocessor=<function preprocess at 0x7e85b71deac0>)),
                ('clf', LogisticRegression(max_iter=1000))])

Evalúo el modelo entrenado. Hago predicciones sobre el conjunto de test y calculo métricas clave: accuracy, precision, recall y F1-score. También muestro la matriz de confusión para ver cuántas reviews positivas y negativas clasificó correctamente y en qué se equivoca.El modelo alcanza un accuracy del 0.84, lo cual indica que clasifica correctamente el 84% de las reviews. Además, la precisión y el recall son similares en ambas clases, lo que significa que el modelo está equilibrado y no favorece demasiado a una clase sobre la otra.



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred_lr = pipeline_lr.predict(X_test)

print("=== Logistic Regression ===\n")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification report:\n")
print(classification_report(y_test, y_pred_lr))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_lr))


=== Logistic Regression ===

Accuracy: 0.839

Classification report:

              precision    recall  f1-score   support

           0       0.83      0.86      0.84       500
           1       0.85      0.82      0.84       500

    accuracy                           0.84      1000
   macro avg       0.84      0.84      0.84      1000
weighted avg       0.84      0.84      0.84      1000

Matriz de confusión:
 [[430  70]
 [ 91 409]]


Defino un segundo modelo usando SVM. Mantengo el mismo preprocesado y vectorización que antes para poder comparar su rendimiento con el de la regresión logística.


In [ ]:
from sklearn.svm import LinearSVC

pipeline_svm = Pipeline([
    ("vect", CountVectorizer(
        preprocessor=preprocess,
        ngram_range=(1, 2),
        max_features=5000
    )),
    ("clf", LinearSVC())
])

pipeline_svm.fit(X_train, y_train)


Pipeline(steps=[('vect',
                 CountVectorizer(max_features=5000, ngram_range=(1, 2),
                                 preprocessor=<function preprocess at 0x7e85b71deac0>)),
                ('clf', LinearSVC())])

En este bloque evalúo el modelo SVM con el mismo conjunto de test. Calculo accuracy, precision, recall y F1-score, además de la matriz de confusión, para comparar su rendimiento con el de la regresión logística.El SVM obtiene un buen rendimiento (accuracy 0.82), pero queda por debajo del modelo de regresión logística. Tiene F1-score ligeramente menor y más errores en la matriz de confusión, por lo que el mejor modelo sigue siendo Logistic Regression.



In [ ]:
y_pred_svm = pipeline_svm.predict(X_test)

print("=== Linear SVM ===\n")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nClassification report:\n")
print(classification_report(y_test, y_pred_svm))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_svm))


=== Linear SVM ===

Accuracy: 0.82

Classification report:

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       500
           1       0.83      0.81      0.82       500

    accuracy                           0.82      1000
   macro avg       0.82      0.82      0.82      1000
weighted avg       0.82      0.82      0.82      1000

Matriz de confusión:
 [[417  83]
 [ 97 403]]


MEJOR MODELO: Logistic Regression